# Homework 1
## Data Analysis
### FINM August Review 

Mark Hendricks

hendricks@uchicago.edu

#### Data

- This homework uses the file, “hw 1 data.xlsx”.
- Find the data in the Github repo associated with the module, (link on Canvas.)

The data file contains...
- Return rates, $r^i_t$, for various asset classes, (via ETFs.)
- Most notable among these securities is SPY, the return on the S&P 500. Denote this as $r^{spy}_t$
- A separate tab gives return rates for a particular portfolio, $r^p_t$

## 1. Regression

1. Estimate the regression of the portfolio return on SPY:
$$r^p_t = \alpha + \beta^{spy} r^{spy}_t + \epsilon^{spy}_t$$
Specifically, report alpha, beta, and the r-squared.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
r_p = pd.read_excel(r'../data/hw_1_data.xlsx', sheet_name='portfolio returns').set_index('Date')
r_i = pd.read_excel(r'../data/hw_1_data.xlsx', sheet_name='security returns').set_index('Date')


1/ create the function display associated to this class
which will display all the features of this class.
example:
you need to print:
order(buy,10,100)

The name of this function is called "display"

In [44]:
X = sm.add_constant(r_i['SPY'])
y = r_p['portfolio']

estimation = sm.OLS(y,X,missing='drop').fit()
estimation.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              portfolio   R-squared:                       0.745
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     455.5
Date:                Wed, 31 Aug 2022   Prob (F-statistic):           3.93e-48
Time:                        00:00:47   Log-Likelihood:                 443.21
No. Observations:                 158   AIC:                            -882.4
Df Residuals:                     156   BIC:                            -876.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0006      0.001     -0.479      0.633      -0.003       0.002
SPY            0.6142      0.029     21.342      0.000       0.557       0.671
==============================================================================
Omnibus:                       21.658   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               65.437
Skew:                           0.448   Prob(JB):                     6.17e-15
Kurtosis:                       6.023   Cond. No.                         24.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

2. Estimate the regression of the portfolio return on SPY and on HYG, the return on high-yield corporate bonds, denoted as $r^{hyg}_t$:
$$r^p_t = \tilde{\alpha} + \tilde{\beta}^{spy} r^{spy}_t + \tilde{\beta}^{hyg}r^{hyg}_t + \tilde{\epsilon}_t$$
Specifically, report alpha, beta, and the r-squared.
Note that the ~ in (2) are just to clarify that these are different from the parameters in (1).

In [45]:
X = sm.add_constant(r_i[['SPY','HYG']])
y = r_p['portfolio']

estimation_multi = sm.OLS(y,X).fit()
estimation_multi.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              portfolio   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.830
Method:                 Least Squares   F-statistic:                     384.2
Date:                Wed, 31 Aug 2022   Prob (F-statistic):           8.57e-61
Time:                        00:00:53   Log-Likelihood:                 476.28
No. Observations:                 158   AIC:                            -946.6
Df Residuals:                     155   BIC:                            -937.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0009      0.001     -0.915      0.362      -0.003       0.001
SPY            0.3843      0.035     11.072      0.000       0.316       0.453
HYG            0.5166      0.058      8.977      0.000       0.403       0.630
==============================================================================
Omnibus:                        1.294   Durbin-Watson:                   2.246
Prob(Omnibus):                  0.524   Jarque-Bera (JB):                1.296
Skew:                           0.214   Prob(JB):                        0.523
Kurtosis:                       2.886   Cond. No.                         66.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

3. Calculate the time-series of fitted regression values:
$$\hat{r}^p_t = \tilde{\alpha} + \tilde{\beta}^{spy} r^{spy}_t + \tilde{\beta}^{hyg}r^{hyg}_t $$
Your statistical package will output these fitted values for you, or you can construct them using the estimated parameters. What is the correlation between $\hat{r}^p_t$ and $r^p_t$ ? How does that compare to the r-squared of the regression in (2)?

In [71]:
print("Correlation is - ",r_p.corrwith(estimation.fittedvalues)[0])
temp = r_p.corrwith(estimation_multi.fittedvalues)[0]
print("R-square value is -",temp**2)


Correlation is -  0.8630624908276984
R-square value is - 0.8321431637628092


4. How do the SPY betas differ across (1) and (2), ($\beta^{spy}$ vs $\tilde{\beta}^{spy}$)? How does this relate to the correlation between $r^{spy}_t$ and $r^{hyg}_t$?

In [57]:
#SPY beta in (2) is smaller than (1) as more of the variance is explained by HYG
r_i['SPY'].corr(r_i['HYG'])

0.7380170112481027

5. Without doing any calculation, would you expect $\epsilon_t$ or $\tilde{\epsilon}_t$ to have higher correlation to $r^{hyg}_t$ ?

In [68]:
print(r_i['HYG'].corr(estimation.resid))
print(f'{r_i["HYG"].corr(estimation_multi.resid):.2f}')

0.3946498277405792
0.00


## 2. Decomposing and Replicating

1. The portfolio return, $r^p_t$ , is a combination of the base assets that are provided here. Use linear regression to uncover which weights were used in constructing the portfolio.
$$r^p_t = \alpha + (\beta^r)^′r_t + \epsilon_t$$

    where $r$ denotes the vector of returns for the individual securities
    - What does the regression find were the original weights?
    - How precise is the estimation? Consider the R-squared and t-stats.


2. Suppose that we want to mimic TIP using the other securities. Run the following regression–but do so only using data through the end of 2018.

$$r^{TIP}_t = \alpha + (\beta)^′r_t + \epsilon_t$$
    
where $r$ denotes the vector of returns for the other securities, excluding TIP.
- Report the r-squared and the estimate of the vector, $\beta$.
- Report the t-stats of the explanatory returns. Which have absolute value greater than 2?
- Plot the returns of TIP along with the replication values.

3. Perhaps the replication results in the previous problem are overstated given that they estimated the parameters within a sample and then evaluated how well the result fit in the same sample. This is known as in-sample fit. Using the estimates through 2018, (the α and β from the previous problem,) calculate the out-of-sample (OOS) values of the replication, using the 2019-2022 returns, denoted r^{oos}_t:
$$\hat{r}^{TIPoos}_t = (\beta)^′r^{oos}_t$$

    - What is the correlation between $\hat{r}^{TIPoos}_t$ and $r^{oos}_t$ ?
    - How does this compare to the $R^2$ from the regression above based on data through 2018?